In [ ]:
import tensorflow as tf
print(tf.__version__)
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import PIL
import pathlib 

2.8.0


In [ ]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

In [ ]:
batch_size = 32
img_height = 180
img_width = 180

In [ ]:
data_dir = "/content/drive/MyDrive/Colab Notebooks/faces" 
data_dir = pathlib.Path(data_dir)
data_dir_test = pathlib.Path("/content/drive/MyDrive/Colab Notebooks/faces_test")

In [ ]:
image_count = len(list(data_dir.glob('*/*.jpg')))
print(image_count)

252


In [ ]:
batch_size = 32
img_height = 180
img_width = 180

In [ ]:
train_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 252 files belonging to 16 classes.
Using 202 files for training.


In [ ]:
val_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir_test,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 62 files belonging to 16 classes.
Using 12 files for validation.


In [ ]:
class_names = train_ds.class_names
print(class_names)

['face1', 'face10', 'face11', 'face12', 'face13', 'face14', 'face15', 'face16', 'face2', 'face3', 'face4', 'face5', 'face6', 'face7', 'face8', 'face9']


In [ ]:
for image_batch, labels_batch in train_ds:
  print(image_batch.shape)
  print(labels_batch.shape)
  break

(32, 180, 180, 3)
(32,)


In [ ]:
normalization_layer = tf.keras.layers.Rescaling(1./255)
normalized_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
image_batch, labels_batch = next(iter(normalized_ds))
first_image = image_batch[0]
# Notice the pixel values are now in `[0,1]`.
print(np.min(first_image), np.max(first_image))

0.029758053 1.0


In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
num_classes = 16

model = tf.keras.Sequential([
  tf.keras.layers.Rescaling(1./255),
  tf.keras.layers.Conv2D(32, 3, activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Conv2D(32, 3, activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Conv2D(32, 3, activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(num_classes)
])

In [ ]:
model.compile(
  optimizer='adam',
  loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=['accuracy'])

In [ ]:
model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=5
)

Epoch 1/5
7/7 [==============================] - 11s 1s/step - loss: 2.8686 - accuracy: 0.0644 - val_loss: 2.7269 - val_accuracy: 0.0833
Epoch 2/5
7/7 [==============================] - 7s 930ms/step - loss: 2.6617 - accuracy: 0.2525 - val_loss: 2.5931 - val_accuracy: 0.1667
Epoch 3/5
7/7 [==============================] - 7s 958ms/step - loss: 2.3400 - accuracy: 0.4208 - val_loss: 2.3298 - val_accuracy: 0.2500
Epoch 4/5
7/7 [==============================] - 7s 947ms/step - loss: 1.7390 - accuracy: 0.5990 - val_loss: 1.6283 - val_accuracy: 0.5833
Epoch 5/5
7/7 [==============================] - 7s 947ms/step - loss: 0.9027 - accuracy: 0.8020 - val_loss: 0.6140 - val_accuracy: 0.8333


In [ ]:
def new_image(test_img_dir):
  data_dir = pathlib.Path(test_img_dir)
  img = tf.keras.utils.load_img(
    data_dir, target_size=(img_height, img_width))
  img_array = tf.keras.utils.img_to_array(img)
  img_array = tf.expand_dims(img_array, 0)
  return img_array
holy_image = new_image("/content/IMG20220212132859.jpg")

In [ ]:
pred = model.predict(holy_image)
score = tf.nn.softmax(pred)

In [ ]:
100 * np.max(score)

90.26066660881042

In [ ]:
class_names[np.argmax(score)]

'face1'

In [ ]:
np.argmax(score)

0

In [ ]:
model.save('my_model.h5')

In [ ]:
model.save('mymodel')

INFO:tensorflow:Assets written to: mymodel/assets
